In [1]:
import os
import torch
from transformers import AutoModel, AutoTokenizer
import torch.nn.functional as F
import hjson

In [2]:
torch.manual_seed(42)

if torch.cuda.is_available():
    device="cuda"
    print(f"[INFO] GPU disponible. Usaremos CUDA y limitaremos a ~3800 MiB de VRAM.")
else:
    device="cpu"
    print("[WARN] GPU no disponible. Usando CPU sin limitación de memoria.")

[INFO] GPU disponible. Usaremos CUDA y limitaremos a ~3800 MiB de VRAM.


In [3]:
model=AutoModel.from_pretrained(
    "../models/jina-reranker-m0",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
    max_memory={
        0: "3000MB"
    }
)
model.eval()

tokenizer=AutoTokenizer.from_pretrained(
    "../models/jina-reranker-m0",
    trust_remote_code=True
)

Some parameters are on the meta device because they were offloaded to the disk.


In [7]:
with open("../assets/tables.hjson", "r") as file:
    tables=hjson.load(file)

table_docs={
    "INVENTARIO_REPORTES": f"{tables['table_reports'][0]}\n{tables['table_reports'][1]}",
    "INVENTARIO_VALIDACIONES": f"{tables['table_valid'][0]}\0{tables['table_valid'][1]}",
    "INVENTARIO_VALIDACIONES_COMPUESTO": f"{tables['table_report_valid'][0]}\n{tables['table_report_valid'][1]}",
    "CLIENTE": f"{tables['table_client'][0]}\n{tables['table_client'][1]}",
    "CONTRATOS": f"{tables['table_contra'][0]}\n{tables['table_contra'][1]}",
    "CONTRATOS_REPORTES": f"{tables['table_contra_report'][0]}\n{tables['table_contra_report'][1]}",
    "CONTRATOS_CLIENTES_REPORTES": f"{tables['table_client_contra_report'][0]}\n{tables['table_client_contra_report'][1]}",
    "GENERALES": f"{tables['general_quest']}",
}
table_desc={
    "INVENTARIO_REPORTES": tables['table_reports'][1],
    "INVENTARIO_VALIDACIONES": tables['table_valid'][1],
    "INVENTARIO_VALIDACIONES_COMPUESTO": tables['table_report_valid'][1],
    "CLIENTE": tables['table_client'][1],
    "CONTRATOS": tables['table_contra'][1],
    "CONTRATOS_REPORTES": tables['table_contra_report'][1],
    "CONTRATOS_CLIENTES_REPORTES": tables['table_client_contra_report'][1],
}

In [8]:
def choose_table(query: str) -> str:
    table_names = list(table_docs.keys())
    pairs = [[query, schema] for schema in table_docs.values()]

    with torch.no_grad():
        # scores quizá sea una lista de floats o un tensor en CPU
        scores = model.compute_score(pairs, max_length=1024, doc_type="text")
    
    # Si scores es un tensor:
    if isinstance(scores, torch.Tensor):
        scores = scores.cpu().tolist()

    # Imprime cada nombre de tabla con su score
    for name, score in zip(table_names, scores):
        print(f"{name}: {score:.4f}")

    # Ahora selecciona el índice del mayor score
    best_idx = int(torch.argmax(torch.tensor(scores)))
    choose=list(table_docs.keys())[best_idx]
    print(f"\nMejor tabla: {choose} (score = {scores[best_idx]:.4f})")

    return [choose]


In [9]:
ejemplos=[
    "Cuales son ?",
]
for q in ejemplos:
    tabla=choose_table(q)
    print(f"→ “{q}”  se refiere más a: {tabla}")

INVENTARIO_REPORTES: 0.7903
INVENTARIO_VALIDACIONES: 0.7643
INVENTARIO_VALIDACIONES_COMPUESTO: 0.7292
CLIENTE: 0.7712
CONTRATOS: 0.7607
CONTRATOS_REPORTES: 0.7535
CONTRATOS_CLIENTES_REPORTES: 0.7292
GENERALES: 0.8191

Mejor tabla: GENERALES (score = 0.8191)
→ “Cuales son ?”  se refiere más a: ['GENERALES']
